<a href="https://colab.research.google.com/github/mimilazarova/DD2424-covid-xray-project/blob/master/playground_diego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
# from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# from tensorflow.python.keras.applications import ResNet50
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

with open('/content/drive/My Drive/DD2424_Project/data/trainX1.pickle', 'rb') as f:
  X1 = pickle.load(f)
with open('/content/drive/My Drive/DD2424_Project/data/trainX2.pickle', 'rb') as f:
  X2 = pickle.load(f)

trainX = np.concatenate((X1, X2))

with open('/content/drive/My Drive/DD2424_Project/data/testX.pickle', 'rb') as f:
  testX = pickle.load(f)
with open('/content/drive/My Drive/DD2424_Project/data/train_y.pickle', 'rb') as f:
  train_y = pickle.load(f)
with open('/content/drive/My Drive/DD2424_Project/data/test_y.pickle', 'rb') as f:
  test_y = pickle.load(f)


In [0]:
N = trainX.shape[0]
testN = testX.shape[0]

trainX = trainX.reshape((N, 224, 224, 1))
testX = testX.reshape((testN, 224, 224, 1))

args = [train_y==0]
X0, vX0, y0, vy0 = train_test_split(trainX[args], train_y[args],  test_size=0.1, random_state=42) 

args = [train_y==1]
X1, vX1, y1, vy1 = train_test_split(trainX[args], train_y[args],  test_size=0.1, random_state=42) 

args = [train_y==2]
X2, vX2, y2, vy2 = train_test_split(trainX[args], train_y[args],  test_size=0.1, random_state=42) 

trainX = np.concatenate((X0, X1, X2))
train_y = np.concatenate((y0, y1, y2))

valX = np.concatenate((vX0, vX1, vX2))
val_y = np.concatenate((vy0, vy1, vy2))

In [0]:
trainX2 = np.copy(trainX)
valX2 = np.copy(valX)
train_y2 = np.copy(train_y)
val_y2 = np.copy(val_y)

In [7]:
trainX2.shape

(12210, 224, 224, 1)

In [8]:
train_y2.shape

(12210,)

In [9]:
valX2.shape

(1359, 224, 224, 1)

In [10]:
val_y2.shape

(1359,)

In [11]:
num_classes = 3
class_weights = {}
for c in range(3):
  class_weights[c] = 1000/np.sum(train_y==c)
  print(class_weights[c])


0.13948946854512484
0.2038735983690112
7.352941176470588


In [0]:
N = trainX.shape[0]
testN = testX.shape[0]

In [13]:
trainX2 = trainX2.reshape((N, 224, 224, 1))
trainX2 = np.repeat(trainX2[0:1000,:,:,:], 3, axis=-1)

trainX2.shape

(1000, 224, 224, 3)

In [14]:
valX2 = valX2.reshape((testN, 224, 224, 1))
valX2 = np.repeat(valX[0:100,:,:,:], 3, axis=-1)
valX2.shape

ValueError: ignored

In [0]:
#Build Model

model1 = Sequential()
model1.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model1.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
model1.layers[0].trainable = False

model1.summary()

In [0]:
#Compile Model
model1.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [0]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
# datagen.fit(trainX)

# fits the model on batches with real-time data augmentation:
# model1.fit(datagen.flow(trainX, train_y, batch_size=32),
                    # steps_per_epoch=len(trainX) / 32, epochs=5)


#Fit model
history1 = model1.fit(trainX, train_y[0:1000], epochs=5, 
                    validation_data=(testX, test_y[0:100]))

In [0]:
model1 = keras.models.Sequential()
model1.add(layers.Conv2D(64, (5, 5), padding = 'same', activation='relu', input_shape=(224, 224, 1)))
model1.add(layers.BatchNormalization())
model1.add(layers.MaxPooling2D((4, 4)))
model1.add(layers.Conv2D(64, (5, 5), padding = 'same', activation='relu'))
model1.add(layers.BatchNormalization())
model1.add(layers.MaxPooling2D((4, 4)))
model1.add(layers.Conv2D(64, (5, 5), activation='relu'))
model1.add(layers.BatchNormalization())
model1.add(layers.Flatten())
model1.add(layers.Dense(64, activation='relu'))
model1.add(layers.Dense(32, activation='relu'))
model1.add(layers.Dense(10, activation='relu'))
model1.add(layers.Dense(3))

model1.summary()

In [0]:
model1.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

history1 = model1.fit(trainX, train_y, epochs=5, 
                    validation_data=(testX, test_y))

In [0]:
def make_model():
  model = keras.models.Sequential()
  model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', input_shape=(224,224,1)))
  model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
  model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())
    
  model.add(layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu', strides=1, padding='same'))

  model.add(layers.Flatten())
  model.add(layers.Dense(32, activation='relu'))
  model.add(BatchNormalization())
  # model.add(Dropout(0.5))
  model.add(layers.Dense(16, activation='relu'))
  model.add(BatchNormalization())
  # model.add(Dropout(0.5))
  model.add(layers.Dense(3))
  return model

In [0]:

model1 = make_model()
model1.summary()

# optimizer = tf.keras.optimizers.Adam (lr=0.001)

model1.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

# reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

history1 = model1.fit(trainX, train_y, epochs=5, 
                    # validation_data=(testX, test_y), callbacks=[reduce_lr])
                    validation_data=(testX, test_y))


In [0]:
plt.plot(history1.history['accuracy'], label='accuracy')
plt.plot(history1.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_y_pred_m = model1.predict(testX)
test_y_pred1 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred1, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)

In [0]:
np.argmax(test_y_pred_m, axis=1)